In [1]:
from setproctitle import setproctitle
setproctitle("prepare")

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

In [2]:
import glob 
import numpy as np
from sunpy.map import Map
from astropy import units as u

/userhome/jeon_mg/miniforge3/envs/zpinn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class PrepareHMI:
    def __init__(self, base_path, hmi_path):
        self.base_path = base_path
        os.makedirs(self.base_path, exist_ok=True)
        self.hmi_path = hmi_path
    
    def load_hmi_data(self, idx1=None, idx2=None, err=False):
        if err:
            hmi_p_files = sorted(glob.glob(os.path.join(self.hmi_path, '*Bp_err.fits'))) 
            hmi_t_files = sorted(glob.glob(os.path.join(self.hmi_path, '*Bt_err.fits')))  
            hmi_r_files = sorted(glob.glob(os.path.join(self.hmi_path, '*Br_err.fits'))) 
        else:
            hmi_p_files = sorted(glob.glob(os.path.join(self.hmi_path, '*Bp.fits'))) 
            hmi_t_files = sorted(glob.glob(os.path.join(self.hmi_path, '*Bt.fits')))  
            hmi_r_files = sorted(glob.glob(os.path.join(self.hmi_path, '*Br.fits'))) 

        if (idx1 is None) or (idx2 is None):
            self.data_paths = list(zip(hmi_p_files, hmi_t_files, hmi_r_files))
        else:
            self.data_paths = list(zip(hmi_p_files, hmi_t_files, hmi_r_files))[idx1:idx2]
        return self.data_paths

    def save_bottom(self, stride=2):
        for hmi_p, hmi_t, hmi_r in self.data_paths:
            obs_date = os.path.basename(hmi_p)[-27:-12]
            b_bottom_path = os.path.join(self.base_path, f'b_bottom_{obs_date}.npy')
            if os.path.exists(b_bottom_path):
                print(f'Exist {b_bottom_path}')
                continue

            p_map = Map(hmi_p)
            t_map = Map(hmi_t)
            r_map = Map(hmi_r)

            maps = [p_map, t_map, r_map]
            maps_binned = [m.superpixel(u.Quantity((stride, stride), u.pixel), func=np.mean) for m in maps]
            B_binned = np.stack([maps_binned[0].data, -maps_binned[1].data, maps_binned[2].data]).transpose()

            b_bottom = np.array(B_binned)

            with open(b_bottom_path, 'wb') as f:
                np.save(f, b_bottom)
            print(f"Saved {b_bottom_path}")

In [10]:
Nz = 160
batch_size = 2500

hmi_path = '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/'
base_path = os.path.expanduser('~/workspace/_data/NOAA12673/')

b_bottom_path = os.path.join(base_path, 'b_bottom')
b_bottom_err_path = os.path.join(base_path, 'b_bottom_err')

os.makedirs(b_bottom_path, exist_ok=True)
os.makedirs(b_bottom_err_path, exist_ok=True)

In [11]:
creater = PrepareHMI(b_bottom_err_path, hmi_path)
creater.load_hmi_data(err=True)

[('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bp_err.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bt_err.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Br_err.fits'),
 ('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bp_err.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bt_err.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Br_err.fits'),
 ('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bp_err.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bt_err.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Br_err.fits'),
 ('/mnt/obsdata/nf2/sdo_A

In [12]:
creater.save_bottom(stride=2)

Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_000000_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_001200_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_002400_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_003600_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_004800_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_010000_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_011200_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_012400_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_013600_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_014800_TAI.npy
Exist /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom_err/b_bottom_0906_020

In [13]:
creater = PrepareHMI(b_bottom_path, hmi_path)
creater.load_hmi_data(err=False)

[('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bp.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bt.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Br.fits'),
 ('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bp.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bt.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Br.fits'),
 ('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bp.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bt.fits',
  '/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Br.fits'),
 ('/mnt/obsdata/nf2/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_c

In [14]:
creater.save_bottom(stride=2)

See https://docs.sunpy.org/en/stable/code_ref/map.html#fixing-map-metadata for how to fix metadata before loading it with sunpy.map.Map.
See https://fits.gsfc.nasa.gov/fits_standard.html forthe FITS unit standards. [sunpy.map.mapbase]


Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_000000.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_001200.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_002400.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_003600.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_004800.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_010000.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_011200.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_012400.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_013600.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_014800.npy
Saved /userhome/jeon_mg/workspace/_data/NOAA12673/b_bottom/b_bottom_20170906_020000.npy
Saved /userhome/jeon_mg/workspac